In [1]:
# install the package
!python -m pip install requests
# import packages
import requests, json, time
from IPython.display import Javascript
from process import Chatbot
import process

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


# Connect to Speakeasy

In [2]:
# url of the speakeasy server
url = "https://speakeasy.ifi.uzh.ch"

# get the api specification
r = requests.get(url + "/client-specs")
spec = json.loads(r.text)

In [3]:
# user login
def login(username: str, password: str):
    return requests.post(url=url + "/api/login", json={"username": username, "password": password})

# check details of the current user
def current(session_token: str):
    return requests.get(url=url + "/api/user/current", params={"session": session_token})

# user logout
def logout(session_token: str):
    return requests.get(url=url + "/api/logout", params={"session": session_token})

In [4]:
# load credentials from a json file
with open("./credentials.json", "r") as f:
    credentials = json.load(f)
username = credentials["agent2"]["username"]
password = credentials["agent2"]["password"]

print("--- log in as an agent")
agent_details = login(username=username, password=password).json()
print("agent details:")
print(json.dumps(agent_details, indent=4))

print("--- check user details with the session token {}".format(agent_details["sessionToken"]))
current_details = current(session_token=agent_details["sessionToken"]).json()
print(json.dumps(current_details, indent=4))

print("--- log out")
r = logout(session_token=agent_details["sessionToken"])
print(r.json())

--- log in as an agent
agent details:
{
    "userDetails": {
        "id": "c3a53f0e-7571-415f-802b-bbdd722fe963",
        "username": "bot_900",
        "role": "BOT"
    },
    "sessionToken": "node018qhgzejj18spu7wsfujfz7yf366296",
    "sessionId": "452c040e-050e-4380-922d-11ccc057e30a",
    "startTime": 1639519201124,
    "userSessionAlias": "inspiring_cohen_66"
}
--- check user details with the session token node018qhgzejj18spu7wsfujfz7yf366296
{
    "userDetails": {
        "id": "c3a53f0e-7571-415f-802b-bbdd722fe963",
        "username": "bot_900",
        "role": "BOT"
    },
    "sessionToken": "node018qhgzejj18spu7wsfujfz7yf366296",
    "sessionId": "452c040e-050e-4380-922d-11ccc057e30a",
    "startTime": 1639519201124,
    "userSessionAlias": "inspiring_cohen_66"
}
--- log out
{'description': 'Logged out'}


# Chat Related Functions

In [5]:
# check available chat rooms
def check_rooms(session_token: str):
    return requests.get(url=url + "/api/rooms", params={"session": session_token})

# check the state of a chat room
def check_room_state(room_id: str, since: int, session_token: str):
    return requests.get(url=url + "/api/room/{}/{}".format(room_id, since), params={"roomId": room_id, "since": since, "session": session_token})

# post a message to a chat room
def post_message(room_id: str, session_token: str, message: str):
    headers = {'Content-Type': 'text/text; charset=utf-8'}
    return requests.post(url=url + "/api/room/{}".format(room_id), params={"roomId": room_id, "session": session_token}, data=message.encode('utf-8'), headers=headers)

In [6]:
process.initialize()

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


Parsing the graph..
Parsing done!
Reading from movies..
Reading movies done!
Reading from humans..
Reading from humans done!
Reading genres..
Reading genres done!
Reading characters..
Reading characters done!
Writing awards..
Writing awards done!


In [7]:
agent_details = login(username=username, password=password).json()
print("--- agent details:")
print(json.dumps(agent_details, indent=4))

chatroom_messages = {}
chatbots = {}
while True:
    current_rooms = check_rooms(session_token=agent_details["sessionToken"]).json()["rooms"]
    print("--- {} chatrooms available".format(len(current_rooms)))

    for idx, room in enumerate(current_rooms):
        room_id = room["uid"]
        print("chat room - {}: {}".format(idx, room_id))

        new_room_state = check_room_state(room_id=room_id, since=0, session_token=agent_details["sessionToken"]).json()
        new_messages = new_room_state["messages"]
        print("found {} messages".format(len(new_messages)))

        if room_id not in chatroom_messages.keys():
            chatroom_messages[room_id] = []
            chatbots[room_id] = Chatbot(room_id)
            response = "Hello, I am your chatbot today. How may I help you? You can write HELP with capital letters to see what kind of questions you can ask me!"
            post_message(room_id=room_id, session_token=agent_details["sessionToken"], message=response)

        if len(chatroom_messages[room_id]) != len(new_messages):
            for message in new_messages:
                if message["ordinal"] >= len(chatroom_messages[room_id]) and message["session"] != agent_details["sessionId"]:
                    try:
                        inform = chatbots[room_id].getWaitingMessage()
                        post_message(room_id=room_id, session_token=agent_details["sessionToken"], message=inform)
                        response = chatbots[room_id].getResponse(message["message"])
                        print('heyyyy ' + response)
                        post_message(room_id=room_id, session_token=agent_details["sessionToken"], message=response)
                    except:
                        response = "Sorry, I did not get that mate. Please rephrase the question for me!"
                        post_message(room_id=room_id, session_token=agent_details["sessionToken"], message=response)

        chatroom_messages[room_id] = new_messages

    time.sleep(1)
    print("")

--- agent details:
{
    "userDetails": {
        "id": "c3a53f0e-7571-415f-802b-bbdd722fe963",
        "username": "bot_900",
        "role": "BOT"
    },
    "sessionToken": "node0jmc9ybly9y6q1jxkf23hyqyri370139",
    "sessionId": "16d58c92-d77e-40c6-b4e7-7e2572077d6b",
    "startTime": 1639519362710,
    "userSessionAlias": "wizardly_swanson_11"
}
--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 0 chatrooms available

--- 1 chatrooms available
chat room - 0: 3995bb71-cbae-46b6-a51e-800d0ec10aa7
found 0 messages

--- 1 chatrooms available
chat room - 0: 3995bb71-cbae-46b6-a51e-800d0ec10aa7
found 1 messages

--- 1 chatrooms available
chat room - 0: 3995bb71-cbae-46b6-a51e-800d0ec10aa7
found 1 messages

--- 1 chatrooms available
chat room - 0: 3995bb71-cbae-46b6-a51e-800d0ec10aa7
found 1 messages

--- 1 chatrooms avai